In [10]:
#in the name of GOD

In [11]:
import pandas as pd 
import numpy as np
from IPython.display import clear_output
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
import matplotlib.cm as cm
import matplotlib as matplotlib

from fastdtw import fastdtw

def color_map_color(value, cmap_name='coolwarm', vmin=0, vmax=10):
    # norm = plt.Normalize(vmin, vmax)
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)  # PiYG
    rgb = cmap(norm(abs(value)))[:3]  # will return rgba, we take only first 3 so we get rgb
    color = matplotlib.colors.rgb2hex(rgb)
    return color
cl=['r','g','b','c','m','y','k']
color=dict()
for i,el in enumerate(cl):
    color.update({i:el})
#extract the labels for clutering precision its for after clustering
def time_convertor(x):
    s,h,m=0,0,0
    s=round(x%60,2)
    m=int(x/60)
    h=int(m/60)
    m=m%60
    time=str(h)+':'+str(m)+':'+str(s)
    return pd.to_datetime('1970-01-01 '+time)


# Display figures inline in Jupyter notebook
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 5)})

In [12]:
#data_read
#prepare Dataset gyroscope
dfg=pd.read_csv('16\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('16\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
df1 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('17\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('17\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
df2 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('20\\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('20\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
df3 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('21\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('21\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
df4 = pd.concat([dfa, dfg], axis=1, join='outer')

In [13]:
label_event_lenght=dict()
#prepare Labeled dataset on 16
dfl=pd.read_csv('16\\groundTruth.csv')
dfl['length']=dfl.en-dfl.st
for event , dft in dfl.groupby('evento'):
    if (event in label_event_lenght):
        label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
    else:
        label_event_lenght.update({event : list(dft.length)})
        
dfl['st_time']=dfl.apply(lambda x:time_convertor(x.st) ,axis=1 )
dfl['en_time']=dfl.apply(lambda x:time_convertor(x.en) ,axis=1 )
dfl=dfl.drop('st',1).drop('en',1).drop('length',1)
evented_label_1=list()
for i in range(len(dfl)):
    evented_label_1.append((dfl.iloc[i][0],dfl.iloc[i][1],dfl.iloc[i][2]))
#prepare Labeled dataset on 17
dfl=pd.read_csv('17\\groundTruth.csv')
dfl['length']=dfl.en-dfl.st
for event , dft in dfl.groupby('evento'):
    if (event in label_event_lenght):
        label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
    else:
        label_event_lenght.update({event : list(dft.length)})

dfl['st_time']=dfl.apply(lambda x:time_convertor(x.st) ,axis=1 )
dfl['en_time']=dfl.apply(lambda x:time_convertor(x.en) ,axis=1 )
dfl=dfl.drop('st',1).drop('en',1).drop('length',1)
evented_label_2=list()
for i in range(len(dfl)):
    evented_label_2.append((dfl.iloc[i][0],dfl.iloc[i][1],dfl.iloc[i][2]))
#prepare Labeled dataset on 20
dfl=pd.read_csv('20\\groundTruth.csv')
dfl['length']=dfl.en-dfl.st
for event , dft in dfl.groupby('evento'):
    if (event in label_event_lenght):
        label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
    else:
        label_event_lenght.update({event : list(dft.length)})

dfl['st_time']=dfl.apply(lambda x:time_convertor(x.st) ,axis=1 )
dfl['en_time']=dfl.apply(lambda x:time_convertor(x.en) ,axis=1 )
dfl=dfl.drop('st',1).drop('en',1).drop('length',1)
evented_label_3=list()
for i in range(len(dfl)):
    evented_label_3.append((dfl.iloc[i][0],dfl.iloc[i][1],dfl.iloc[i][2]))
#prepare Labeled dataset on 21
dfl=pd.read_csv('21\\groundTruth.csv')
dfl['length']=dfl.en-dfl.st
for event , dft in dfl.groupby('evento'):
    if (event in label_event_lenght):
        label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
    else:
        label_event_lenght.update({event : list(dft.length)})
dfl['st_time']=dfl.apply(lambda x:time_convertor(x.st) ,axis=1 )
dfl['en_time']=dfl.apply(lambda x:time_convertor(x.en) ,axis=1 )
dfl=dfl.drop('st',1).drop('en',1).drop('length',1)
evented_label_4=list()
for i in range(len(dfl)):
    evented_label_4.append((dfl.iloc[i][0],dfl.iloc[i][1],dfl.iloc[i][2]))

In [14]:
eve1=pd.DataFrame(evented_label_1)
eve1[3]=['evented_label_1' for x in evented_label_1]

eve2=pd.DataFrame(evented_label_2)
eve2[3]=['evented_label_2' for x in evented_label_2]

eve3=pd.DataFrame(evented_label_3)
eve3[3]=['evented_label_3' for x in evented_label_3]

eve4=pd.DataFrame(evented_label_4)
eve4[3]=['evented_label_4' for x in evented_label_4]

evented_label_All=pd.concat([eve1, eve2,eve3,eve4])

In [15]:
evented_label_All

,0,1,2,3
0,evento_nao_agressivo,1970-01-01 00:00:02.000,1970-01-01 00:00:06.500,evented_label_1
1,curva_direita_agressiva,1970-01-01 00:00:19.500,1970-01-01 00:00:23.500,evented_label_1
2,evento_nao_agressivo,1970-01-01 00:00:30.000,1970-01-01 00:00:33.500,evented_label_1
3,curva_direita_agressiva,1970-01-01 00:01:35.000,1970-01-01 00:01:38.000,evented_label_1
4,curva_esquerda_agressiva,1970-01-01 00:04:07.000,1970-01-01 00:04:11.500,evented_label_1
...,...,...,...,...
17,aceleracao_agressiva,1970-01-01 00:09:25.000,1970-01-01 00:09:27.800,evented_label_4
18,aceleracao_agressiva,1970-01-01 00:11:40.000,1970-01-01 00:11:43.200,evented_label_4
19,aceleracao_agressiva,1970-01-01 00:11:54.000,1970-01-01 00:11:57.000,evented_label_4
20,evento_nao_agressivo,1970-01-01 00:12:31.500,1970-01-01 00:12:34.700,evented_label_4


In [16]:
rate=(df2.index[6]-df3.index[5]).total_seconds()
window_list=list()
for event in label_event_lenght:
    window_list.append((event,round(max(label_event_lenght[event])/rate)))
window_list

[('curva_direita_agressiva', 225),
 ('curva_esquerda_agressiva', 225),
 ('evento_nao_agressivo', 225),
 ('troca_faixa_direita_agressiva', 125),
 ('aceleracao_agressiva', 245),
 ('freada_agressiva', 185),
 ('troca_faixa_esquerda_agressiva', 120)]

In [17]:
#supervised
window={226:['curva_direita_agressiva','curva_esquerda_agressiva','aceleracao_agressiva'],
        126:['troca_faixa_direita_agressiva','troca_faixa_esquerda_agressiva'],
        186:['freada_agressiva']
       }

In [18]:
#Dont RUN this part
#we should optimize M
#lengh of random event
import random
random.seed(123)

def TempEvent_genarator(df,L,i):
    M=round(random.gauss(0.55,0.5)*L/2)
    temp_event=list()
    pointer=0
    stop=False
    #go on timeserise for event extraction
    while stop!=True:
        clear_output(wait=True)
        pointer=pointer + M
        l_min=pointer-int(L/2)
        l_max=pointer+int(L/2)
        if l_max<0:
            l_max=L/2
        if (l_min<0):
            l_min=0
        if (l_max>len(df)):
            l_max=len(df)
            stop=True
        print('extraction ',l_max/len(df),' % of ',i,' is complete')
        temp_event.append(df[l_min:l_max])
    return temp_event


temp_event=list()
for l,DF in enumerate([df1,df2,df3,df4]):
    te=list()
    for i,lw in enumerate(window):
        te.append((window[lw],TempEvent_genarator(DF,lw,i+1)))
    temp_event.append(te)

with open("temp_event.txt", "wb") as fp:
    pickle.dump(temp_event, fp)

extraction  1.0  % of  3  is complete


In [21]:
#calculate tempevnt with labdeled event
def calculate_distance(temp_event,evented_label):
    Dum=evented_label.groupby(0).size()
    n=0
    k=0
    for L , AEV in temp_event:
        for i in L: 
            k=k+len(AEV)*Dum[i]

    EVENT_DS=dict()
    for L , AEV in temp_event:
        for eve_name in L:
            Y=list()
            for EV in AEV:
                X=list()
                for eve,dfe in evented_label.groupby(0):
                    if eve==eve_name:
                        for el in dfe.iloc:
                            clear_output(wait=True)
                            n=n+1
                            print('disstance calculatoin ',round((n/k)*100,4),' %  is complete')
                            if   el[3]=='evented_label_1':
                                X.append(fastdtw(EV,df1[el[1]:el[2]])[0])
                            elif el[3]=='evented_label_2':
                                X.append(fastdtw(EV,df2[el[1]:el[2]])[0])
                            elif el[3]=='evented_label_3':
                                X.append(fastdtw(EV,df3[el[1]:el[2]])[0])
                            elif el[3]=='evented_label_4':
                                X.append(fastdtw(EV,df4[el[1]:el[2]])[0])
                Y.append(X)
            EVENT_DS.update({eve_name:Y})
    return EVENT_DS


t1=time.time()
EVENT_DS=list()
for TE in temp_event:
    EVENT_DS.append(calculate_distance(TE,evented_label_All))

t2=time.time()
with open("Disstance.txt", "wb") as fp:
    pickle.dump(EVENT_DS, fp)

print(round((t2-t1)/60) ,'min has time for calculaton distant')

disstance calculatoin  100.0  %  is complete
241 min has time for calculaton distant


In [22]:
with open("temp_event.txt", "rb") as fp:
    temp_event = pickle.load(fp)
with open("Disstance.txt", "rb") as fp:
    EVENT_DS = pickle.load(fp)

In [23]:
def has_overlap(A_start, A_end, B_start, B_end):
    latest_start = max(A_start, B_start)
    earliest_end = min(A_end, B_end)
    return latest_start <= earliest_end
def export_labels(EVENT_DS,temp_event,evented_label):
    Labels=dict()
    for name in EVENT_DS:
        if name in {x[0] for x in evented_label}:
            label=list()
            for i in range(0,3):
                if name in temp_event[i][0]:
                    el=[(x[1],x[2]) for x in evented_label if x[0]==name]                    
                    for te in temp_event[i][1]:
                        sta =te.index[0]
                        ena =te.index[-1]
                        D=False
                        for stb,enb in el:
                            if has_overlap(sta,ena,stb,enb):
                                D=True
                        if D==False:
                            label.append(False)
                        else:
                            label.append(True)
            Labels.update({name:label})
    return Labels

def export_dataframe(ED,evented_label):
    Dt=dict()
    for name in ED:
        if name in {x[0] for x in evented_label}:
            df=pd.DataFrame(ED[name])
            Dt.update({name:df})
    return Dt

#prepare each data with label
Labeled_Event=[evented_label_1,evented_label_2,evented_label_3,evented_label_4]
Data_Frame=list()
for i in range(4):
    Data_dict=export_dataframe(EVENT_DS[i],Labeled_Event[i])
    Label_dict=export_labels( EVENT_DS[i],temp_event[i],Labeled_Event[i])
    for name in Data_dict :
        Data_dict[name]['label']=Label_dict[name]
    Data_Frame.append(Data_dict)


#prepare as data frame    
feature_name=list(pd.core.common.flatten(window.values()))
Train_Data={x:None for x in feature_name}

for name in feature_name:
    for df in Data_Frame:
        if name in df:
            if type(Train_Data[name])==None:
                Train_Data[name]=df[name]
            else:
                Train_Data[name]=pd.concat([df[name],Train_Data[name]], ignore_index=True)


In [27]:
Train_Data['curva_direita_agressiva']

,0,1,2,3,4,5,6,7,8,9,10,label
0,1062.015535,952.436789,920.127561,1096.316454,995.629507,1143.001716,1110.641003,1311.701689,1163.363101,1534.527044,1375.345204,False
1,1094.586953,965.045920,944.507193,1115.913814,1020.728878,1162.351957,1132.215259,1320.968916,1165.415199,1569.512715,1407.334926,False
2,1147.217060,992.942614,978.602362,1141.387087,1048.371460,1195.109873,1142.754726,1341.778559,1191.336390,1567.514577,1452.186938,False
3,1183.879516,1035.486601,1022.961876,1170.632577,1081.989275,1245.865721,1143.940665,1329.952034,1201.132482,1554.718536,1479.840019,False
4,1208.393546,1076.152739,1080.891942,1211.080681,1116.779633,1297.881611,1143.872470,1330.538820,1211.034507,1574.589316,1506.577674,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3056,1135.905282,1064.879507,1093.688986,1136.422309,1074.254382,1337.926759,1265.584614,1485.265977,1195.881876,1495.278540,1401.147746,False
3057,1137.813669,1088.500072,1085.658285,1131.980198,1083.279235,1337.672591,1266.083956,1482.139479,1197.413229,1489.821675,1400.849971,False
3058,1147.196106,1063.992134,1091.683291,1123.775901,1084.130372,1346.251035,1266.800915,1470.657234,1210.925569,1486.199961,1405.359836,False
3059,1158.574733,1097.363589,1079.547271,1136.357523,1100.892220,1348.180709,1285.598047,1484.681353,1212.264628,1493.099443,1412.263085,False


In [ ]:
#Dont Run this part
from copy import deepcopy
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report

NUMBER_OF_TEST=100
Model=dict()
Report_of_Acc=dict()
for name in Train_Data:
    estimator = Id3Estimator(max_depth=3, min_samples_split=1, prune=True,
                gain_ratio=True, min_entropy_decrease=0, is_repeating=True)
    Data=Train_Data[name].drop('label',1)
    label=Train_Data[name].label
    temp=list()
    for NT in range(NUMBER_OF_TEST):
        estimator.fit(Data,label , check_input=True)
        ACC=accuracy_score(y_true=label,y_pred=estimator.predict(Data))
        temp.append((ACC,deepcopy(estimator)))
    temp_index=[(x[0],i) for i,x in enumerate(temp)]
    temp_index.sort()
    print('Accuracy Score -',name,' : ',temp[temp_index[-1][1]][0])
    estimator=temp[temp_index[-1][1]][1]
    Model.update({name:deepcopy(estimator)})
    Report_of_Acc.update({name:classification_report(label, estimator.predict(Data))})
with open("Model.txt", "wb") as fp:
    pickle.dump(Model, fp)
with open("Report.txt", "wb") as fp:
    pickle.dump(Report_of_Acc, fp)

In [10]:
from copy import deepcopy
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report

with open("Model.txt", "rb") as fp:
    Model = pickle.load(fp)
with open("Report.txt", "rb") as fp:
    Report_of_Acc = pickle.load(fp)

In [11]:
estimator=Model['curva_direita_agressiva']
print(Report_of_Acc['curva_direita_agressiva'])
print(export_text(estimator.tree_))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99      3265
        True       0.96      0.59      0.73       120

    accuracy                           0.98      3385
   macro avg       0.97      0.80      0.86      3385
weighted avg       0.98      0.98      0.98      3385


4 <=913.94: True (37) 
4 >913.94
|   1 <=1353.63
|   |   5 <=1048.73: True (3) 
|   |   5 >1048.73: False (2253/25) 
|   1 >1353.63
|   |   6 <=1237.27: True (12) 
|   |   6 >1237.27: False (33/6) 



In [12]:
import re
def export_tree_item(x):
    x=export_text(estimator.tree_)
    rep=re.compile(r'(\d) >(\d+.\d+)')
    mo=rep.findall(x)
    return [(int(x1),float(x2)) for x1,x2 in mo]

Model_param=dict()
for name in Model.keys():
    df=[x[1] for x in evented_label_All.groupby(0) if x[0] ==name ][0]
    estimator=Model[name]
    X=dict()
    for dtree_eve , threshold in export_tree_item(export_text(estimator.tree_)) :
        el=df.iloc[dtree_eve]
        if   el[3]=='evented_label_1':
            X.update({dtree_eve:(threshold,df1[el[1]:el[2]])})
        elif el[3]=='evented_label_2':
            X.update({dtree_eve:(threshold,df2[el[1]:el[2]])})
        elif el[3]=='evented_label_3':
            X.update({dtree_eve:(threshold,df3[el[1]:el[2]])})
        elif el[3]=='evented_label_4':
            X.update({dtree_eve:(threshold,df4[el[1]:el[2]])})
    Model_param.update({name:X})

with open("Model_param.txt", "wb") as fp:
    pickle.dump(Model_param, fp)
